In [1]:
import numpy as np
import pandas as pd

In [2]:
offline = False

In [3]:
data = pd.read_csv('WIFI_AP_Passenger_Records_chusai_1stround.csv')
data.columns = ['wifi_ap_tag','passenger_count','time_stamp']

In [4]:
data[:1]

,wifi_ap_tag,passenger_count,time_stamp
0,E1-1A-1<E1-1-01>,15,2016-09-10-18-55-04


In [5]:
print len(data['wifi_ap_tag'].drop_duplicates())
print len(data)
print min(data['time_stamp'])
print max(data['time_stamp'])

749
4133492
2016-09-10-18-55-04
2016-09-14-14-59-01


In [6]:
data[data['time_stamp'] < '2016-09-10-18-55-04']

,wifi_ap_tag,passenger_count,time_stamp


In [7]:
wifi_ap_tag_list = list(data['wifi_ap_tag'].drop_duplicates())
hour_list = ['15','16','17']
slice_list = ['0','1','2','3','4','5'] 
if offline:
    time = '2016-09-13-'
else:
    time = '2016-09-14-'

In [8]:
predict_df = []
for i in range(len(wifi_ap_tag_list)):
    for j in range(len(hour_list)):
        for k in range(len(slice_list)):
            predict_df.append([wifi_ap_tag_list[i],time+hour_list[j]+'-'+slice_list[k]])
predict_df = pd.DataFrame(data = predict_df,columns = ['WIFIAPTag','slice10min'])

In [9]:
predict_df[:5]

,WIFIAPTag,slice10min
0,E1-1A-1<E1-1-01>,2016-09-14-15-0
1,E1-1A-1<E1-1-01>,2016-09-14-15-1
2,E1-1A-1<E1-1-01>,2016-09-14-15-2
3,E1-1A-1<E1-1-01>,2016-09-14-15-3
4,E1-1A-1<E1-1-01>,2016-09-14-15-4


In [10]:
print len(predict_df)
print 749*3*6

13482
13482


In [11]:
#基本可以确定wifi_ap_tag恒定
data_tmp = data[data['time_stamp'] < '2016-09-13']
print max(data_tmp['time_stamp'])
print len(data_tmp['wifi_ap_tag'].drop_duplicates())

2016-09-12-23-59-01
749


In [12]:
def get_slice_10_min(data):
    str_time_stamp = data.split('-')
    slice_tmp = int(str_time_stamp[4])
    slice = 0
    if slice_tmp >= 50:
        slice = 5
    elif slice_tmp >= 40:
        slice = 4
    elif slice_tmp >= 30:
        slice = 3
    elif slice_tmp >= 20:
        slice = 2
    elif slice_tmp >= 10:
        slice = 1
    else:
        slice = 0
    return str(str_time_stamp[0])+'-'+str(str_time_stamp[1])+'-'+str(str_time_stamp[2])+'-'+str(str_time_stamp[3])+'-'+str(slice)

In [13]:
data['slice_10_min'] = data['time_stamp'].apply(lambda x : get_slice_10_min(x))

In [14]:
data[:1]

,wifi_ap_tag,passenger_count,time_stamp,slice_10_min
0,E1-1A-1<E1-1-01>,15,2016-09-10-18-55-04,2016-09-10-18-5


In [15]:
avg_slice_10_min = data[['wifi_ap_tag','passenger_count','slice_10_min']].groupby(['wifi_ap_tag','slice_10_min']).sum().reset_index()

In [16]:
avg_slice_10_min[:1]

,wifi_ap_tag,slice_10_min,passenger_count
0,E1-1A-1<E1-1-01>,2016-09-10-18-5,81


In [17]:
avg_slice_10_min['passenger_count'] = avg_slice_10_min['passenger_count'].apply(lambda x : float(x)/10)

In [18]:
avg_slice_10_min[:1]

,wifi_ap_tag,slice_10_min,passenger_count
0,E1-1A-1<E1-1-01>,2016-09-10-18-5,8.1


In [19]:
def get_pre_10_min(data):
    data_tmp = data.split('-')
    year = str(data_tmp[0])
    month = str(data_tmp[1])
    day = int(data_tmp[2])
    hour = int(data_tmp[3])
    slice = int(data_tmp[4])
    
    if slice >= 1:
        slice = slice - 1
    elif hour >= 1:
        slice = 5
        hour = hour - 1
    else:
        slice = 5
        hour = 23
        day = day - 1

    day = str(day) if day >= 10 else '0'+str(day)
    hour = str(hour) if hour >= 10 else '0'+str(hour)
    
    return year+'-'+month+'-'+day+'-'+hour+'-'+str(slice)

In [20]:
#测试用例
get_pre_10_min('2016-09-10-00-0')

'2016-09-09-23-5'

In [21]:
#predict_df['pre_10_min'] = predict_df['slice10min'].apply(lambda x : get_pre_10_min(x))
#predict_df['pre_20_min'] = predict_df['pre_10_min'].apply(lambda x : get_pre_10_min(x))
#predict_df['pre_30_min'] = predict_df['pre_20_min'].apply(lambda x : get_pre_10_min(x))
if offline:
    predict_df['pre_10_min'] = '2016-09-13-14-5'
    predict_df['pre_20_min'] = '2016-09-13-14-4'
    predict_df['pre_30_min'] = '2016-09-13-14-3'
    predict_df['pre_40_min'] = '2016-09-13-14-2'
    predict_df['pre_50_min'] = '2016-09-13-14-1'
    predict_df['pre_60_min'] = '2016-09-13-14-0'
else:
    predict_df['pre_10_min'] = '2016-09-14-14-5'
    predict_df['pre_20_min'] = '2016-09-14-14-4'
    predict_df['pre_30_min'] = '2016-09-14-14-3'
    predict_df['pre_40_min'] = '2016-09-14-14-2'
    predict_df['pre_50_min'] = '2016-09-14-14-1'
    predict_df['pre_60_min'] = '2016-09-14-14-0'

In [22]:
predict_df[:5]

,WIFIAPTag,slice10min,pre_10_min,pre_20_min,pre_30_min,pre_40_min,pre_50_min,pre_60_min
0,E1-1A-1<E1-1-01>,2016-09-14-15-0,2016-09-14-14-5,2016-09-14-14-4,2016-09-14-14-3,2016-09-14-14-2,2016-09-14-14-1,2016-09-14-14-0
1,E1-1A-1<E1-1-01>,2016-09-14-15-1,2016-09-14-14-5,2016-09-14-14-4,2016-09-14-14-3,2016-09-14-14-2,2016-09-14-14-1,2016-09-14-14-0
2,E1-1A-1<E1-1-01>,2016-09-14-15-2,2016-09-14-14-5,2016-09-14-14-4,2016-09-14-14-3,2016-09-14-14-2,2016-09-14-14-1,2016-09-14-14-0
3,E1-1A-1<E1-1-01>,2016-09-14-15-3,2016-09-14-14-5,2016-09-14-14-4,2016-09-14-14-3,2016-09-14-14-2,2016-09-14-14-1,2016-09-14-14-0
4,E1-1A-1<E1-1-01>,2016-09-14-15-4,2016-09-14-14-5,2016-09-14-14-4,2016-09-14-14-3,2016-09-14-14-2,2016-09-14-14-1,2016-09-14-14-0


In [23]:
def get_pre_day_slice(data):
    data_tmp = data.split('-')
    year = str(data_tmp[0])
    month = str(data_tmp[1])
    day = int(data_tmp[2])
    hour = str(data_tmp[3])
    slice = str(data_tmp[4])
    
    day = day - 1
    
    day = str(day) if day >= 10 else '0'+str(day)
    
    return year+'-'+month+'-'+day+'-'+hour+'-'+slice

In [24]:
predict_df['pre_1_day'] = predict_df['slice10min'].apply(lambda x : get_pre_day_slice(x))
predict_df['pre_2_day'] = predict_df['pre_1_day'].apply(lambda x : get_pre_day_slice(x))
if not offline:
    predict_df['pre_3_day'] = predict_df['pre_2_day'].apply(lambda x : get_pre_day_slice(x))

In [25]:
predict_df[:5]

,WIFIAPTag,slice10min,pre_10_min,pre_20_min,pre_30_min,pre_40_min,pre_50_min,pre_60_min,pre_1_day,pre_2_day,pre_3_day
0,E1-1A-1<E1-1-01>,2016-09-14-15-0,2016-09-14-14-5,2016-09-14-14-4,2016-09-14-14-3,2016-09-14-14-2,2016-09-14-14-1,2016-09-14-14-0,2016-09-13-15-0,2016-09-12-15-0,2016-09-11-15-0
1,E1-1A-1<E1-1-01>,2016-09-14-15-1,2016-09-14-14-5,2016-09-14-14-4,2016-09-14-14-3,2016-09-14-14-2,2016-09-14-14-1,2016-09-14-14-0,2016-09-13-15-1,2016-09-12-15-1,2016-09-11-15-1
2,E1-1A-1<E1-1-01>,2016-09-14-15-2,2016-09-14-14-5,2016-09-14-14-4,2016-09-14-14-3,2016-09-14-14-2,2016-09-14-14-1,2016-09-14-14-0,2016-09-13-15-2,2016-09-12-15-2,2016-09-11-15-2
3,E1-1A-1<E1-1-01>,2016-09-14-15-3,2016-09-14-14-5,2016-09-14-14-4,2016-09-14-14-3,2016-09-14-14-2,2016-09-14-14-1,2016-09-14-14-0,2016-09-13-15-3,2016-09-12-15-3,2016-09-11-15-3
4,E1-1A-1<E1-1-01>,2016-09-14-15-4,2016-09-14-14-5,2016-09-14-14-4,2016-09-14-14-3,2016-09-14-14-2,2016-09-14-14-1,2016-09-14-14-0,2016-09-13-15-4,2016-09-12-15-4,2016-09-11-15-4


In [26]:
predict_df['pre_1_10_min'] = predict_df['pre_1_day'].apply(lambda x : get_pre_10_min(x))

avg_slice_10_min_copy = avg_slice_10_min.copy()
avg_slice_10_min_copy.columns = ['WIFIAPTag','pre_1_10_min','pre_1_10_passenger_count']
predict_df = pd.merge(left=predict_df,right=avg_slice_10_min_copy,on=['WIFIAPTag','pre_1_10_min'],how='left',sort=False).fillna(0)

In [27]:
avg_slice_10_min_copy = avg_slice_10_min.copy()
avg_slice_10_min_copy.columns = ['WIFIAPTag','pre_10_min','pre_10_passenger_count']
predict_df = pd.merge(left=predict_df,right=avg_slice_10_min_copy,on=['WIFIAPTag','pre_10_min'],how='left',sort=False).fillna(0)

In [28]:
avg_slice_10_min_copy = avg_slice_10_min.copy()
avg_slice_10_min_copy.columns = ['WIFIAPTag','pre_20_min','pre_20_passenger_count']
predict_df = pd.merge(left=predict_df,right=avg_slice_10_min_copy,on=['WIFIAPTag','pre_20_min'],how='left',sort=False).fillna(0)

In [29]:
avg_slice_10_min_copy = avg_slice_10_min.copy()
avg_slice_10_min_copy.columns = ['WIFIAPTag','pre_30_min','pre_30_passenger_count']
predict_df = pd.merge(left=predict_df,right=avg_slice_10_min_copy,on=['WIFIAPTag','pre_30_min'],how='left',sort=False).fillna(0)

In [30]:
avg_slice_10_min_copy = avg_slice_10_min.copy()
avg_slice_10_min_copy.columns = ['WIFIAPTag','pre_40_min','pre_40_passenger_count']
predict_df = pd.merge(left=predict_df,right=avg_slice_10_min_copy,on=['WIFIAPTag','pre_40_min'],how='left',sort=False).fillna(0)

In [31]:
avg_slice_10_min_copy = avg_slice_10_min.copy()
avg_slice_10_min_copy.columns = ['WIFIAPTag','pre_50_min','pre_50_passenger_count']
predict_df = pd.merge(left=predict_df,right=avg_slice_10_min_copy,on=['WIFIAPTag','pre_50_min'],how='left',sort=False).fillna(0)

In [32]:
avg_slice_10_min_copy = avg_slice_10_min.copy()
avg_slice_10_min_copy.columns = ['WIFIAPTag','pre_60_min','pre_60_passenger_count']
predict_df = pd.merge(left=predict_df,right=avg_slice_10_min_copy,on=['WIFIAPTag','pre_60_min'],how='left',sort=False).fillna(0)

In [33]:
avg_slice_10_min_copy = avg_slice_10_min.copy()
avg_slice_10_min_copy.columns = ['WIFIAPTag','pre_1_day','pre_1_passenger_count']
predict_df = pd.merge(left=predict_df,right=avg_slice_10_min_copy,on=['WIFIAPTag','pre_1_day'],how='left',sort=False).fillna(0)

In [34]:
avg_slice_10_min_copy = avg_slice_10_min.copy()
avg_slice_10_min_copy.columns = ['WIFIAPTag','pre_2_day','pre_2_passenger_count']
predict_df = pd.merge(left=predict_df,right=avg_slice_10_min_copy,on=['WIFIAPTag','pre_2_day'],how='left',sort=False).fillna(0)

In [35]:
if not offline:
    avg_slice_10_min_copy = avg_slice_10_min.copy()
    avg_slice_10_min_copy.columns = ['WIFIAPTag','pre_3_day','pre_3_passenger_count']
    predict_df = pd.merge(left=predict_df,right=avg_slice_10_min_copy,on=['WIFIAPTag','pre_3_day'],how='left',sort=False).fillna(0)

In [36]:
predict_df[:2]

,WIFIAPTag,slice10min,pre_10_min,pre_20_min,pre_30_min,pre_40_min,pre_50_min,pre_60_min,pre_1_day,pre_2_day,...,pre_1_10_passenger_count,pre_10_passenger_count,pre_20_passenger_count,pre_30_passenger_count,pre_40_passenger_count,pre_50_passenger_count,pre_60_passenger_count,pre_1_passenger_count,pre_2_passenger_count,pre_3_passenger_count
0,E1-1A-1<E1-1-01>,2016-09-14-15-0,2016-09-14-14-5,2016-09-14-14-4,2016-09-14-14-3,2016-09-14-14-2,2016-09-14-14-1,2016-09-14-14-0,2016-09-13-15-0,2016-09-12-15-0,...,13.6,2.8,1.9,3.8,1.0,0.6,3.9,16.4,4.2,3.2
1,E1-1A-1<E1-1-01>,2016-09-14-15-1,2016-09-14-14-5,2016-09-14-14-4,2016-09-14-14-3,2016-09-14-14-2,2016-09-14-14-1,2016-09-14-14-0,2016-09-13-15-1,2016-09-12-15-1,...,16.4,2.8,1.9,3.8,1.0,0.6,3.9,16.7,6.0,4.2


In [37]:
#测试
if offline:
    print predict_df[(predict_df['slice10min'] > '2016-09-13-15-0') & (predict_df['slice10min'] < '2016-09-13-15-3')]['slice10min'].drop_duplicates()
else:
    print predict_df[(predict_df['slice10min'] > '2016-09-14-15-0') & (predict_df['slice10min'] < '2016-09-14-15-3')]['slice10min'].drop_duplicates()

1    2016-09-14-15-1
2    2016-09-14-15-2
Name: slice10min, dtype: object


In [38]:
def get_score(y,pred_y):
    score = 0.0
    for i in range(len(y)):
        score += (y[i]-pred_y[i])*(y[i]-pred_y[i])
    return score

In [39]:
if offline:
    predict_df.loc[(predict_df['slice10min'] == '2016-09-13-15-0'),'passengerCount'] = 0.8*((predict_df['pre_10_passenger_count']+predict_df['pre_20_passenger_count']+predict_df['pre_30_passenger_count']+predict_df['pre_40_passenger_count']+predict_df['pre_50_passenger_count']+predict_df['pre_60_passenger_count'])/6)+0.2*((predict_df['pre_1_passenger_count']+predict_df['pre_2_passenger_count'])/2)
    predict_df.loc[((predict_df['slice10min'] >= '2016-09-13-15-1') & (predict_df['slice10min'] < '2016-09-13-16-3')),'passengerCount'] = 0.6*((predict_df['pre_10_passenger_count']+predict_df['pre_20_passenger_count']+predict_df['pre_30_passenger_count'])/3)+0.4*(0.6*predict_df['pre_1_passenger_count']+0.4*predict_df['pre_2_passenger_count'])
    #predict_df.loc[(predict_df['slice10min'] == '2016-09-13-15-1'),'passengerCount'] = 0.7*((predict_df['pre_10_passenger_count']+predict_df['pre_20_passenger_count']+predict_df['pre_30_passenger_count'])/3)+0.3*(0.6*predict_df['pre_1_passenger_count']+0.4*predict_df['pre_2_passenger_count'])
    #predict_df.loc[(predict_df['slice10min'] == '2016-09-13-15-1'),'passengerCount'] = 0.65*((predict_df['pre_10_passenger_count']+predict_df['pre_20_passenger_count']+predict_df['pre_30_passenger_count'])/3)+0.35*(0.6*predict_df['pre_1_passenger_count']+0.4*predict_df['pre_2_passenger_count'])
    predict_df.loc[((predict_df['slice10min'] >= '2016-09-13-15-3') & (predict_df['slice10min'] < '2016-09-13-16-0')),'passengerCount'] = 0.55*((predict_df['pre_10_passenger_count']+predict_df['pre_20_passenger_count']+predict_df['pre_30_passenger_count'])/3)+0.45*(0.6*predict_df['pre_1_passenger_count']+0.4*predict_df['pre_2_passenger_count'])
    predict_df.loc[((predict_df['slice10min'] >= '2016-09-13-16-0') & (predict_df['slice10min'] < '2016-09-13-16-3')),'passengerCount'] = 0.5*((predict_df['pre_10_passenger_count']+predict_df['pre_20_passenger_count']+predict_df['pre_30_passenger_count'])/3)+0.5*(0.6*predict_df['pre_1_passenger_count']+0.4*predict_df['pre_2_passenger_count'])
    predict_df.loc[((predict_df['slice10min'] >= '2016-09-13-16-3') & (predict_df['slice10min'] < '2016-09-13-17-0')),'passengerCount'] = 0.45*((predict_df['pre_10_passenger_count']+predict_df['pre_20_passenger_count']+predict_df['pre_30_passenger_count'])/3)+0.55*(0.6*predict_df['pre_1_passenger_count']+0.4*predict_df['pre_2_passenger_count'])
    predict_df.loc[((predict_df['slice10min'] >= '2016-09-13-17-0') & (predict_df['slice10min'] < '2016-09-13-17-3')),'passengerCount'] = 0.4*((predict_df['pre_10_passenger_count']+predict_df['pre_20_passenger_count']+predict_df['pre_30_passenger_count'])/3)+0.6*(0.6*predict_df['pre_1_passenger_count']+0.4*predict_df['pre_2_passenger_count'])
    predict_df.loc[(predict_df['slice10min'] >= '2016-09-13-17-3'),'passengerCount'] = 0.35*((predict_df['pre_10_passenger_count']+predict_df['pre_20_passenger_count']+predict_df['pre_30_passenger_count'])/3)+0.65*(0.6*predict_df['pre_1_passenger_count']+0.4*predict_df['pre_2_passenger_count'])
else:
    #线上 11,6813
    #predict_df.loc[(predict_df['slice10min'] < '2016-09-14-15-3'),'passengerCount'] = predict_df['pre_10_passenger_count']
    #predict_df.loc[((predict_df['slice10min'] >= '2016-09-14-15-3') & (predict_df['slice10min'] < '2016-09-14-16-0')),'passengerCount'] = 0.4*predict_df['pre_10_passenger_count']+0.3*predict_df['pre_20_passenger_count']+0.3*predict_df['pre_30_passenger_count']
    #predict_df.loc[((predict_df['slice10min'] >= '2016-09-14-16-0') & (predict_df['slice10min'] < '2016-09-14-17-0')),'passengerCount'] = 0.6*((predict_df['pre_10_passenger_count']+predict_df['pre_20_passenger_count']+predict_df['pre_30_passenger_count'])/3)+0.4*((predict_df['pre_1_passenger_count']+predict_df['pre_2_passenger_count']+predict_df['pre_3_passenger_count'])/3)
    #predict_df.loc[(predict_df['slice10min'] >= '2016-09-14-17-0'),'passengerCount'] = 0.4*((predict_df['pre_10_passenger_count']+predict_df['pre_20_passenger_count']+predict_df['pre_30_passenger_count'])/3)+0.6*((predict_df['pre_1_passenger_count']+predict_df['pre_2_passenger_count']+predict_df['pre_3_passenger_count'])/3)
    
    #时间弄错了
    #线上 9,9819     pre_x_day 5 : 3 : 2    
    #线上 100894     pre_x_day 5 : 4 : 1 
    #线上 99431      pre_x_day 4 : 4 : 2
    predict_df.loc[(predict_df['slice10min'] >= '2016-09-13-17-3'),'passengerCount'] = 0.35*((predict_df['pre_10_passenger_count']+predict_df['pre_20_passenger_count']+predict_df['pre_30_passenger_count'])/3)+0.65*(0.4*predict_df['pre_1_passenger_count']+0.4*predict_df['pre_2_passenger_count']+0.2*predict_df['pre_3_passenger_count'])
    
    #one change 98387
    predict_df.loc[(predict_df['slice10min'] == '2016-09-14-15-0'),'passengerCount'] = predict_df['pre_10_passenger_count']
    
    #two change 98145
    predict_df.loc[(predict_df['slice10min'] == '2016-09-14-15-1'),'passengerCount'] = 0.5*predict_df['pre_10_passenger_count']+0.5*(0.4*predict_df['pre_1_passenger_count']+0.4*predict_df['pre_2_passenger_count']+0.2*predict_df['pre_3_passenger_count'])
    
    #线上   跟上面差不多
    #predict_df.loc[(predict_df['slice10min'] == '2016-09-14-15-0'),'passengerCount'] = 0.8*predict_df['pre_10_passenger_count']+0.2*(0.5*predict_df['pre_1_passenger_count']+0.3*predict_df['pre_2_passenger_count']+0.2*predict_df['pre_3_passenger_count'])
    
    #predict_df.loc[(predict_df['slice10min'] == '2016-09-14-15-1'),'passengerCount'] = 0.7*((predict_df['pre_10_passenger_count']+predict_df['pre_20_passenger_count']+predict_df['pre_30_passenger_count'])/3)+0.3*(0.5*predict_df['pre_1_passenger_count']+0.3*predict_df['pre_2_passenger_count']+0.2*predict_df['pre_3_passenger_count'])
    #predict_df.loc[(predict_df['slice10min'] == '2016-09-14-15-2'),'passengerCount'] = 0.65*((predict_df['pre_10_passenger_count']+predict_df['pre_20_passenger_count']+predict_df['pre_30_passenger_count'])/3)+0.35*(0.5*predict_df['pre_1_passenger_count']+0.3*predict_df['pre_2_passenger_count']+0.2*predict_df['pre_3_passenger_count'])
    #predict_df.loc[((predict_df['slice10min'] >= '2016-09-13-15-1') & (predict_df['slice10min'] < '2016-09-13-16-3')),'passengerCount'] = 0.6*((predict_df['pre_10_passenger_count']+predict_df['pre_20_passenger_count']+predict_df['pre_30_passenger_count'])/3)+0.4*(0.4*predict_df['pre_1_passenger_count']+0.4*predict_df['pre_2_passenger_count']+0.2*predict_df['pre_3_passenger_count'])
    
    #predict_df.loc[((predict_df['slice10min'] >= '2016-09-14-15-3') & (predict_df['slice10min'] < '2016-09-14-16-0')),'passengerCount'] = 0.55*((predict_df['pre_10_passenger_count']+predict_df['pre_20_passenger_count']+predict_df['pre_30_passenger_count'])/3)+0.45*(0.5*predict_df['pre_1_passenger_count']+0.3*predict_df['pre_2_passenger_count']+0.2*predict_df['pre_3_passenger_count'])
    #predict_df.loc[((predict_df['slice10min'] >= '2016-09-14-16-0') & (predict_df['slice10min'] < '2016-09-14-16-3')),'passengerCount'] = 0.5*((predict_df['pre_10_passenger_count']+predict_df['pre_20_passenger_count']+predict_df['pre_30_passenger_count'])/3)+0.5*(0.5*predict_df['pre_1_passenger_count']+0.3*predict_df['pre_2_passenger_count']+0.2*predict_df['pre_3_passenger_count'])
    #predict_df.loc[((predict_df['slice10min'] >= '2016-09-14-16-3') & (predict_df['slice10min'] < '2016-09-14-17-0')),'passengerCount'] = 0.45*((predict_df['pre_10_passenger_count']+predict_df['pre_20_passenger_count']+predict_df['pre_30_passenger_count'])/3)+0.55*(0.5*predict_df['pre_1_passenger_count']+0.3*predict_df['pre_2_passenger_count']+0.2*predict_df['pre_3_passenger_count'])
    #predict_df.loc[((predict_df['slice10min'] >= '2016-09-14-17-0') & (predict_df['slice10min'] < '2016-09-14-17-3')),'passengerCount'] = 0.4*((predict_df['pre_10_passenger_count']+predict_df['pre_20_passenger_count']+predict_df['pre_30_passenger_count'])/3)+0.6*(0.5*predict_df['pre_1_passenger_count']+0.3*predict_df['pre_2_passenger_count']+0.2*predict_df['pre_3_passenger_count'])
    #predict_df.loc[(predict_df['slice10min'] >= '2016-09-14-17-3'),'passengerCount'] = 0.35*((predict_df['pre_10_passenger_count']+predict_df['pre_20_passenger_count']+predict_df['pre_30_passenger_count'])/3)+0.65*(0.5*predict_df['pre_1_passenger_count']+0.3*predict_df['pre_2_passenger_count']+0.2*predict_df['pre_3_passenger_count'])

In [40]:
result = pd.DataFrame()
result['passengerCount'] = predict_df['passengerCount']
result['WIFIAPTag'] = predict_df['WIFIAPTag']
result['slice10min'] = predict_df['slice10min']

In [41]:
if offline:
    avg_slice_10_min_copy = avg_slice_10_min.copy()
    avg_slice_10_min_copy.columns = ['WIFIAPTag','slice10min','passenger_count']
    offline_result = pd.merge(left=result,right=avg_slice_10_min_copy,on=['WIFIAPTag','slice10min'],how='left',sort=False).fillna(0)
    print get_score(offline_result['passenger_count'],offline_result['passengerCount'])
else:
    import os
    if not os.path.exists('submit'):
        os.makedirs('submit')
    result.to_csv('./submit/airport_gz_passenger_predict.csv',index=None)

In [ ]:
#比较同一wifi_ap_tag下，对应不同天的slice_10_min,passenger_cont的情况
avg_slice_10_min[:1]

In [ ]:
#趋势
predict_df[['pre_1_passenger_count','pre_2_passenger_count']].plot()
import matplotlib.pyplot as plt
plt.show()